In [62]:
import pygame
from pygame.locals import *
import sys,random, os

In [63]:
fps = 30
fClock = pygame.time.Clock()
pygame.init()
ws, hs = pygame.display.Info().current_w, pygame.display.Info().current_h
print(f'Your Resolution: {ws} x {hs}')
width = 288
height = 512
baseY = height * 0.8
pX = width/10
pY = height/2
screen=pygame.display.set_mode((width,height))
pygame.display.set_caption('SD FlappyBird')

player = pygame.image.load('flappy/gallery/images/bird.png').convert_alpha()
bg = pygame.image.load('flappy/gallery/images/background.png').convert_alpha()
pipe = (     pygame.transform.rotate ( pygame.image.load('flappy/gallery/images/pipe.png').convert_alpha(), 180), 
            pygame.image.load( 'flappy/gallery/images/pipe.png').convert_alpha()   )   
            # Upperpipe: pipe[0]
msg = pygame.image.load( 'flappy/gallery/images/message.png').convert_alpha()
start=pygame.image.load( 'flappy/gallery/images/start.png').convert_alpha()
base = pygame.image.load( 'flappy/gallery/images/base.png').convert_alpha()
no = ( pygame.image.load( 'flappy/gallery/images/0.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/1.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/2.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/3.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/4.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/5.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/6.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/7.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/8.png').convert_alpha(),
pygame.image.load( 'flappy/gallery/images/9.png').convert_alpha()          
)

screen.blit(bg,(0,0))
screen.blit(player,(pX,pY))
screen.blit(msg,(width/5,height/5))
screen.blit(start,(width/3.5,height/2))
screen.blit(base,(0,baseY))
pygame.display.update()
fClock.tick(fps)


Your Resolution: 1920 x 1080


410

In [64]:
def welcome():
    for event in pygame.event.get():
        if (event == QUIT) or (event.type==KEYDOWN and event.key==K_ESCAPE):
            pygame.quit()
            sys.exit()
        elif event.type==KEYDOWN and (event.key== K_SPACE or event.key==K_UP):
            game()


In [65]:
def game():
    baseY = height * 0.8
    pX = width/10
    pY = height/2
    
    newpipe1=getrandom()
    newpipe2=getrandom()

    upperpipes = [      { 'x':width,'y':newpipe1[0]['y'] },
                        { 'x':width*2,'y':newpipe2[0]['y'] }      ]


    lowerpipes = [      { 'x':width,'y':newpipe1[1]['y'] },
                        { 'x':width*2,'y':newpipe2[1]['y'] }      ]

    pipeVelX = -5
    pVelY = -8
    pMinVelY = -9
    pMaxVelY = 10
    pAccY   = 1

    flapped = False
    pFlapVelY = -8
    score=0
    while True:
        for event in pygame.event.get():
            if (event == QUIT) or (event.type==KEYDOWN and event.key==K_ESCAPE):
                pygame.quit()
                sys.exit()
            elif event.type==KEYDOWN and (event.key== K_SPACE or event.key==K_UP):
                if pY>0:
                    flapped=True
                    pVelY = pFlapVelY
                    sounds['wing'].play()

        # Collision/Out
        crashTest = isCollide ( pX, pY, upperpipes, lowerpipes)
        if crashTest:
            sounds['die'].play()
            break
        
        # Score
        pMid = pX + player.get_width()/2
        for up in upperpipes:
            pipeMid = up['x'] + pipe[0].get_width()/2
            if pipeMid <= pMid < pipeMid + 5:
                score +=1
                print('Your Score: ', score)
                sounds['point'].play()
        
        if pVelY < pMaxVelY and not flapped:
            pVelY += pAccY
        
        # Flapped Once?
        if flapped:
            flapped=False

        # Moving Pipes to left
        for up,lp in zip(upperpipes, lowerpipes):
            up['x'] += pipeVelX 
            lp['x'] += pipeVelX

        # Add new ones
            if 0 < up['x'] < 5:
                newpipe = getrandom()
                upperpipes.append(newpipe[0])
                lowerpipes.append(newpipe[1])
        
        # Remove from left
            if up['x'] <= -( pipe[0].get_width() ):
                upperpipes.pop(0)
                lowerpipes.pop(0)
        
        pHeight = player.get_height()
        pY = pY + min(pVelY, baseY - pY - pHeight)

        # Blitting (frame by frame)
        screen.blit (bg,     (0,0))
        for up, lp  in zip(upperpipes, lowerpipes):
            screen.blit (pipe[0], (up['x'], up['y']))
            screen.blit (pipe[1], (lp['x'], lp['y']))
        screen.blit (player, (pX,pY))
        screen.blit (base,   (0,baseY))

        # score print
        x=0
        digits = [ int(x) for x in list(str(score)) ]
        w = 0
        Xoffset = 0
        for d in digits:
            w += no[d].get_width()
        Xoffset = (width - w)/2

        for d in digits:
            screen.blit(no[d], (Xoffset, height*0.9))
            Xoffset += no[d].get_width()
        
        pygame.display.update()
        fClock.tick(fps)


In [66]:
def getrandom():
    offset = player.get_height() + 70
    
    # Lower Pipe
    y2 = offset + random.randrange (0, int(height - base.get_height() - 1.2*offset))
    
    # Upperpipe
    x = width + 200
    y1 = - ( pipe[0].get_height() + offset - y2 ) 
    
    result = [
        { 'x': x, 'y': y1 },
        { 'x': x, 'y': y2 },
    ]
    
    return result

In [67]:
def isCollide(pX, pY, upperpipes, lowerpipes):
    
    # Sky
    if pY < 0:
        return True

    # Upperpipe collision
    for up in upperpipes:
        if ( abs(pX - up['x']) < pipe[0].get_width() and pY <= up['y'] + pipe[0].get_height()):
            return True

    pY = pY + player.get_height()
    # Ground
    if pY >= baseY:
        return True
    
    # LowerPipe
    for lp in lowerpipes:
        if (abs(pX - lp['x']) < pipe[1].get_width() and pY >= lp['y']):
            return True

In [61]:
if __name__=='__main__': 
    sounds=[]
    sounds = {     'die': pygame.mixer.Sound('flappy/gallery/audio/die.wav'),
    'point':pygame.mixer.Sound('flappy/gallery/audio/point.wav')    }
    sounds['swoosh'] = pygame.mixer.Sound('flappy/gallery/audio/swoosh.wav')
    sounds['wing'] = pygame.mixer.Sound('flappy/gallery/audio/wing.wav')

    while True:
        welcome()

SystemExit: 